In [1]:
import time
from datetime import datetime
from os import mkdir
from os.path import exists, join

import pandas as pd
import numpy as np
import scipy as sp

import psutil

In [2]:
if not exists('logs'):
    mkdir(log_dir)

In [3]:
now = datetime.now()
log_dir = join('logs', now.strftime('%d-%H'))
if not exists(log_dir):
    mkdir(log_dir)
log_dir

'logs/05-08'

In [4]:
worker_dirs = ['train'] + ['test']

In [5]:
feture_extract_path = ['extract_hotel_feature', 'extract_basic_room_feature',
                      'extract_hotel_room_feature', 'extract_room_feature',
                      'extract_uid_feture']

In [6]:
# feture_extract_path = ['extract_order_feature']

In [7]:
# feture_combine_path = ['combine_sample']
feture_combine_path = ['extract_order_feature']

In [8]:
max_run = 4

In [9]:
def get_args(feture_extract_path, worker_dirs):
    for p in feture_extract_path:
        for d in worker_dirs:
            yield [p, d]

In [10]:
def check(p):
    pm = p.memory_info()
    is_zero = all(map(lambda x:getattr(pm, x)==0, ['rss','vms', 'shared', 'text', 'lib', 'data', 'dirty']))
    return is_zero

In [11]:
def is_wait(pool, max_pool=4):
    if len(pool) < max_pool:
        print(datetime.now(), 'pool size', len(pool))
        return False
    else:
        time.sleep(10)
        not_run_p = []
        for i, p in enumerate(pool[:]):
            not_run = check(p)
            print(datetime.now(), 'process', i, 'not runing' if not_run else 'run')
            if not_run:
                not_run_p.append(i)
        not_run_p.sort(reverse=True)
        for i in not_run_p:
            p = pool.pop(i)
            p.kill()
        return True

In [12]:
def run_args(extract_args, max_pool=4):
    pool = []
    for p, n in extract_args:
        print(datetime.now(), 'begin', p, n)
        fn = join(log_dir, '{}_{}.txt'.format(p, n))
        pool.append(psutil.Popen(['python', p+'.py', n], stdout=open(fn, 'w')))
        while is_wait(pool, max_pool):
            print(datetime.now(), 'run in ', p, n)
    while is_wait(pool, 1):
        print('wait for pool over')

In [13]:
# for p, n in extract_args:
#     print(datetime.now(), 'begin', p, n)
#     fn = join(log_dir, '{}_{}.txt'.format(p, n))
#     pool.append(psutil.Popen(['python', p+'.py', n], stdout=open(fn, 'w')))
#     while is_wait(pool):
#         print(datetime.now(), 'run in ', p, n)

In [14]:
extract_args = get_args(feture_extract_path, worker_dirs)
combine_args = get_args(feture_combine_path, worker_dirs)

In [15]:
run_args(extract_args, max_run)

2017-08-05 08:52:30.592402 begin extract_hotel_feature train
2017-08-05 08:52:30.598291 pool size 1
2017-08-05 08:52:30.598392 begin extract_hotel_feature test
2017-08-05 08:52:30.603747 pool size 2
2017-08-05 08:52:30.603826 begin extract_basic_room_feature train
2017-08-05 08:52:30.609969 pool size 3
2017-08-05 08:52:30.610077 begin extract_basic_room_feature test
2017-08-05 08:52:40.631900 process 0 run
2017-08-05 08:52:40.632135 process 1 run
2017-08-05 08:52:40.632252 process 2 run
2017-08-05 08:52:40.632363 process 3 run
2017-08-05 08:52:40.632418 run in  extract_basic_room_feature test
2017-08-05 08:52:50.642969 process 0 not runing
2017-08-05 08:52:50.643384 process 1 not runing
2017-08-05 08:52:50.643891 process 2 not runing
2017-08-05 08:52:50.644517 process 3 not runing
2017-08-05 08:52:50.646239 run in  extract_basic_room_feature test
2017-08-05 08:52:50.646338 pool size 0
2017-08-05 08:52:50.646402 begin extract_hotel_room_feature train
2017-08-05 08:52:50.653799 pool size

In [16]:
run_args(combine_args, max_run)